In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from glob import glob

In [4]:
import itertools

#### some pandas opts

In [5]:
pd.options.display.max_columns = 100

In [6]:
pd.options.display.max_rows = 500

#### where are the files?

In [7]:
%ls /home/dizak/Pulpit/BIONAS/G148/SNPs_calling/

16 documents from consensus.geneious*
16 documents from WT-day70 to WT-zero point.geneious*
BY-day0/
BY-nup133-day70/
BY-WT-day70/
consensus/
mapped_contigs/
mapped_reads/
nup mapped to wt cons.geneious*
W303-cog7-day42/
W303-nup133-day42/
WT-day70/
WT-day70.xlsx*


In [8]:
W303_cog7_day42_C1_files = glob("/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/W303-cog7-day42/C1/*csv")

In [9]:
W303_cog7_day42_C2_files = glob("/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/W303-cog7-day42/C2/*csv")

In [10]:
W303_cog7_day42_C3_files = glob("/home/dizak/Pulpit/BIONAS/G148/SNPs_calling/W303-cog7-day42/C3/*csv")

#### get non-redundant list of genes in the inputfiles

In [11]:
def find_flat_value(inputfiles_list,
                    col_name = "CDS"):
    """
    Get flat list of desired values from list of CSV files.
    
    Parameters
    -------
    inputfiles_list: list of str
        List of input CSV files.
    col_name: str
        Desired column name in the input CSV file.
    
    Returns
    -------
    list of desired values.
    """
    values_list = []
    for i in inputfiles_list:
        df = pd.read_csv(i)
        if len(df) == 0:
            pass
        elif col_name not in df.columns:
            pass
        else:
            values_list.append(df[col_name].dropna().drop_duplicates().tolist())
    return list(itertools.chain.from_iterable(values_list))

#### get values from input files by key

In [12]:
def find_by_key(inputfiles_list,
                key):
    """
    Get pandas.DataFrame selected by a given key from list of CSV files.
    
    Parameters
    -------
    inputfiles_list: list of str
        List of input CSV files.
    key: str, int, float, bool
        Key used as query against rows in the CSV files.
    value_col: str
        Column name which holds values to be returned.
    
    Returns
    -------
    dict of lists of desired values if pandas.Dataframe not empty
    None if pandas.DataFrame empty
    """
    values_list = []
    for i in inputfiles_list:
        filename = "".join(i.split("/")[-1].split(".")[:-1])
        df = pd.read_csv(i)
        if len(df) == 0:
            pass
        else:
            if isinstance(key, str) == True:
                df_dtype_sel = df.select_dtypes(include=["object"])
            elif isinstance(key, int) == True:
                df_dtype_sel = df.select_dtypes(include=["int"])
            elif isinstance(key, float) == True:
                df_dtype_sel = df.select_dtypes(include=["float"])
            elif isinstance(key, bool) == True:
                df_dtype_sel = df.select_dtypes(include=["bool"])
            else:
                raise ValueError("key must str, int, float or bool dtype")
            for col in df_dtype_sel.columns:
                df_sel = df[df_dtype_sel[col] == key]
                if len(df_sel) > 0:
                    return {"dataframe": df_sel,
                            "filename": filename}

#### get whole set of pandas.DataFrames selections in one dict

In [345]:
def get_dfs_set(key_list,
                files_list,
                vals=["Minimum",
                      "Maximum",
                      "Change"],
                df_key="dataframe",
                index_by_key=True):
    """
    Get desired values in pandas.Dataframe gathered in dict by
    the list of keys.
    """
    out_dict = {}
    for i in key_list:
        key_vals = find_by_key(files_list,
                              key=i)[df_key]
        out_dict[i] = key_vals[vals]
    if index_by_key is True:
        for i in out_dict:
            out_dict[i].index = [i] * len(out_dict[i])
    else:
        pass
    return out_dict

In [440]:
def get_dfs_set(key_list,
                files_list,
                vals=["Minimum",
                      "Maximum",
                      "Change"],
                df_key="dataframe",
                key_index_name="Gene",
                smpl_index_name="Sample",
                row_index_name="Number",
                index_by_key=True,
                smpl_index_val=None,):
    """
    Get desired values in pandas.Dataframe gathered in dict by
    the list of keys.
    """
    out_dict = {}
    for i in key_list:
        key_vals = find_by_key(files_list,
                              key=i)[df_key]
        out_dict[i] = key_vals[vals]
    if index_by_key is True:
        for i in out_dict:
            key_index = [i] * len(out_dict[i])
            if smpl_index_val is not None:
                smpl_index = [smpl_index_val] * len(out_dict[i])
                tpls = list(zip(*[key_index,
                                  smpl_index,
                                  out_dict[i].index]))
                mindex = pd.MultiIndex.from_tuples(tpls,
                                                   names=[key_index_name,
                                                          smpl_index_name,
                                                          row_index_name])
            else:
                tpls = list(zip(*[key_index,
                                  out_dict[i].index]))
                mindex = pd.MultiIndex.from_tuples(tpls,
                                                   names=[key_index_name,
                                                          row_index_name])
            out_dict[i].index = mindex
    return out_dict

In [441]:
W303_cog7_day42_C1_dfs = get_dfs_set(W303_cog7_day42_C1_CDSs,
                                     W303_cog7_day42_C1_files,
                                     smpl_index_val="C1")

In [442]:
dfs_list = [W303_cog7_day42_C1_dfs[i] for i in W303_cog7_day42_C1_dfs.keys()]

In [443]:
merged_dfs = reduce(lambda df1, df2: pd.concat([df1, df2],
                                              axis=0),
                    dfs_list)

In [444]:
C1_merged_dfs = merged_dfs.sort(columns=["Minimum", "Maximum"])

/home/dizak/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [445]:
C1_merged_dfs

Minimum  Maximum  \
Gene                       Sample Number                     
COS7 CDS                   C1     35         1933     1933   
                                  34         1946     1946   
                                  33         1948     1948   
                                  32         1979     1979   
                                  31         2009     2009   
                                  30         2011     2011   
                                  29         2016     2016   
                                  28         2019     2019   
                                  27         2025     2025   
                                  26         2028     2029   
                                  25         2031     2031   
                                  24         2036     2036   
                                  23         2038     2038   
                                  22         2041     2041   
                                  21         2046     2046   
                                  20         2049     2049   
                                  19         2052     2053   
PAU11 CDS                  C1     268        6414     6414   
                                  267        6417     6417   
                                  266        6420     6420   
                                  265        6423     6423   
                                  264        6429     6431   
                                  263        6435     6435   
                                  262        6440     6440   
                                  261        6442     6442   
                                  260        6445     6445   
                                  259        6449     6449   
                                  258        6455     6455   
                                  257        6462     6464   
                                  256        6466     6466   
                                  255        6470     6470   
                                  254        6473     6473   
                                  253        6477     6477   
                                  252        6483     6483   
                                  251        6486     6488   
                                  250        6490     6490   
                                  249        6502     6502   
                                  248        6511     6512   
                                  247        6519     6519   
                                  246        6524     6526   
                                  245        6529     6530   
                                  244        6532     6532   
                                  243        6537     6537   
                                  242        6541     6543   
                                  241        6547     6548   
                                  240        6550     6550   
                                  239        6552     6552   
                                  238        6557     6558   
YPS5 CDS                   C1     237        8630     8632   
                                  236        8648     8650   
                                  235        8651     8652   
                                  234        8655     8656   
                                  233        8660     8660   
PAU14 CDS                  C1     8          8870     8870   
TDA8 CDS                   C1     307       13617    13617   
                                  306       13655    13655   
hypothetical protein CDS   C1     7         25590    25590   
FLO9 CDS                   C1     288       25956    25956   
                                  287       25967    25968   
                                  286       25969    25970   
                                  285       25973    25973   
                                  284       25977    25977   
                                  283       25981    25983   
            

In [397]:
W303_cog7_day42_C2_dfs = get_dfs_set(W303_cog7_day42_C2_CDSs, W303_cog7_day42_C2_files)

In [398]:
dfs_list = [W303_cog7_day42_C2_dfs[i] for i in W303_cog7_day42_C2_dfs.keys()]

In [399]:
merged_dfs = reduce(lambda df1, df2: pd.concat([df1, df2],
                                              axis=0),
                    dfs_list)

In [400]:
C2_merged_dfs = merged_dfs.sort(columns=["Minimum", "Maximum"])

/home/dizak/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [407]:
C2_merged_dfs

Minimum  Maximum  \
Gene                       Number                     
COS7 CDS                   42         1933     1933   
                           41         1946     1946   
                           40         1948     1948   
                           39         1979     1979   
                           38         2009     2009   
                           37         2011     2011   
                           36         2016     2016   
                           35         2019     2019   
                           34         2025     2025   
                           33         2028     2029   
                           32         2031     2031   
                           31         2036     2036   
                           30         2038     2038   
                           29         2041     2041   
PAU11 CDS                  301        6414     6414   
                           300        6417     6419   
                           299        6423     6423   
                           298        6440     6440   
                           297        6442     6442   
                           296        6445     6445   
                           295        6449     6449   
                           294        6455     6455   
                           293        6462     6464   
                           292        6466     6466   
                           291        6470     6470   
                           290        6473     6473   
                           289        6477     6477   
                           288        6483     6483   
                           287        6486     6488   
                           286        6490     6490   
                           285        6502     6502   
                           284        6511     6512   
                           283        6519     6519   
                           282        6524     6526   
                           281        6529     6530   
                           280        6532     6532   
                           279        6537     6537   
                           278        6541     6543   
                           277        6547     6546   
                           276        6554     6554   
                           275        6557     6558   
COS8 CDS                   55         6844     6844   
                           54         6850     6850   
                           53         6857     6857   
                           52         6865     6865   
                           51         6913     6913   
YPS5 CDS                   274        8644     8644   
                           273        8650     8650   
                           272        8655     8656   
                           271        8660     8660   
PAU14 CDS                  8          8870     8870   
TDA8 CDS                   333       13617    13617   
                           332       13655    13655   
FLO9 CDS                   321       25956    25956   
                           320       25963    25965   
                           319       25967    25970   
                           318       25973    25973   
                           317       25977    25979   
                           316       25981    25983   
                           315       25985    25984   
                           314       25987    25987   
                           313       25990    25992   
                           312       25993    25995   
                           311       25998    26000   
                           310       26002    26003   
                           309       26005    26007   
                           308       26009    26010   
                           307       26011    26012   
                           306       26014    26019   
                           305       26021    26025   
                           304       26271    26271   
                    

In [405]:
pd.merge(left=C1_merged_dfs, right=C2_merged_dfs)

,Minimum,Maximum,Change
0,1933,1933,T -> A
1,1946,1946,T -> G
2,1948,1948,A -> C
3,1979,1979,A -> G
4,2009,2009,A -> G
5,2011,2011,T -> C
6,2016,2016,A -> G
7,2019,2019,A -> G
8,2025,2025,G -> A
9,2028,2029,TA -> CT


#### merge any given number of dfs

In [68]:
def merge_dfs(dfs,
              sort_cols=["Minimum",
                         "Maximum"],
              reconstr_index=True):
    """
    Merge any number of pandas.DataFrame into one.
    Indexes must be identical in all the pandas.DataFrames.
    
    Parameters
    -------
    dfs: list
        list of pandas.DataFrames to merge.
    sort_cols: list, None
        list of col names to sort the final
        pandas.DataFrame by. No sorting if None.
    """
    for x in [set(i.index) for i in dfs]:
        assert len(x) == 1, "Indices are not homogenic."
    new_index = list(x)
    df = reduce(lambda df1, df2: pd.merge(left=df1,
                                          right=df2,
                                          how="outer"),
                dfs)
    if sort_cols is not None:
        df.sort(columns=sort_cols)
    if reconstr_index is True:
        df.index = len(df) * new_index
    return df

#### let's find out which CDS are present in all the files one by one

In [15]:
W303_cog7_day42_C1_CDSs = find_flat_value(W303_cog7_day42_C1_files)
W303_cog7_day42_C2_CDSs = find_flat_value(W303_cog7_day42_C2_files)
W303_cog7_day42_C3_CDSs = find_flat_value(W303_cog7_day42_C3_files)

#### let's gather some info about each of the CDS from each sample

In [16]:
W303_cog7_day42_C1_dfs = get_dfs_set(W303_cog7_day42_C1_CDSs, W303_cog7_day42_C1_files)
W303_cog7_day42_C2_dfs = get_dfs_set(W303_cog7_day42_C2_CDSs, W303_cog7_day42_C2_files)
W303_cog7_day42_C3_dfs = get_dfs_set(W303_cog7_day42_C3_CDSs, W303_cog7_day42_C3_files)

#### which CDSs are common for all 3?

In [39]:
W303_cog7_day42_comm_CDS = reduce(lambda x, y: [i for i in x if i in y],
                                  [W303_cog7_day42_C1_CDSs,
                                   W303_cog7_day42_C2_CDSs,
                                   W303_cog7_day42_C3_CDSs])

In [69]:
out_dict = {}
for i in W303_cog7_day42_comm_CDS:
    out_dict[i] = merge_dfs([W303_cog7_day42_C1_dfs[i],
                             W303_cog7_day42_C2_dfs[i],
                             W303_cog7_day42_C3_dfs[i]])

/home/dizak/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:25: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [275]:
df = W303_cog7_day42_C1_dfs["FLO1 CDS"]

In [276]:
df.index = range(len(df))

In [277]:
sample = len(W303_cog7_day42_C1_dfs["FLO1 CDS"]) * ["C1"]

In [278]:
gene = len(W303_cog7_day42_C1_dfs["FLO1 CDS"]) * ["FLO1 CDS"]

In [279]:
tpls = list(zip(*[sample, gene, range(len(df))]))

In [280]:
df.index = pd.MultiIndex.from_tuples(tpls, names=["sample", "gene", "number"])

In [281]:
df

Minimum  Maximum            Change
sample gene     number                                    
C1     FLO1 CDS 0        204538   204538            A -> G
                1        204529   204529            G -> A
                2        204459   204460          WA -> TT
                3        204456   204456            A -> T
                4        204453   204454          CG -> TT
                5        204449   204451        CAC -> GTT
                6        204443   204448  TSSCCA -> GAAACC
                7        204435   204435            G -> C
                8        204431   204433        ACM -> CAA
                9        204428   204430       BHB -> CGGT
                10       204424   204426        RMS -> CCA
                11       204419   204419            C -> A
                12       204410   204411          CA -> AT
                13       204407   204408          CH -> AT
                14       204404   204404            C -> A
                15       204402   204402            K -> G
                16       204398   204400        CAY -> GCT
                17       204396   204396            A -> C
                18       204394   204394            C -> A
                19       204388   204388            C -> A
                20       203986   203986            A -> T
                21       203977   203977            C -> T
                22       203679   203680          TG -> AA
                23       203677   203677            C -> T

In [282]:
df2 = W303_cog7_day42_C2_dfs["FLO1 CDS"]

In [283]:
df2.index = range(len(df2))

In [284]:
sample = len(W303_cog7_day42_C2_dfs["FLO1 CDS"]) * ["C2"]

In [285]:
gene = len(W303_cog7_day42_C2_dfs["FLO1 CDS"]) * ["FLO1 CDS"]

In [286]:
tpls = list(zip(*[sample, gene, range(len(df2))]))

In [287]:
df2.index = pd.MultiIndex.from_tuples(tpls, names=["sample", "gene", "number"])

In [288]:
df2

Minimum  Maximum      Change
sample gene     number                              
C2     FLO1 CDS 0        204538   204538      A -> G
                1        204529   204529      G -> A
                2        204459   204460    WA -> TT
                3        204456   204456      A -> T
                4        204453   204454    CG -> TT
                5        204439   204441  GWG -> ACT
                6        204434   204436  CGA -> AAC
                7        204431   204433  ACM -> GTC
                8        204428   204430  BHB -> ACG
                9        204416   204418  MMY -> TCA
                10       204413   204414    AM -> CC
                11       204410   204412  CAY -> ATA
                12       204407   204408    CH -> AT
                13       204404   204404      C -> A
                14       204402   204402      K -> G
                15       204398   204400  CAY -> GCT
                16       204396   204396      A -> C
                17       204394   204394      C -> A
                18       204388   204388      C -> A
                19       203986   203986      A -> T
                20       203977   203977      C -> T
                21       203713   203713      A -> T
                22       203705   203705      G -> A
                23       203696   203698  ATC -> GTT
                24       203679   203680    TG -> AA
                25       203677   203677      C -> T

In [289]:
df2.T

sample        C2                                                              \
gene    FLO1 CDS                                                               
number        0       1         2       3         4           5           6    
Minimum   204538  204529    204459  204456    204453      204439      204434   
Maximum   204538  204529    204460  204456    204454      204441      204436   
Change    A -> G  G -> A  WA -> TT  A -> T  CG -> TT  GWG -> ACT  CGA -> AAC   

sample                                                                       \
gene                                                                          
number           7           8           9         10          11        12   
Minimum      204431      204428      204416    204413      204410    204407   
Maximum      204433      204430      204418    204414      204412    204408   
Change   ACM -> GTC  BHB -> ACG  MMY -> TCA  AM -> CC  CAY -> ATA  CH -> AT   

sample                                                                       \
gene                                                                          
number       13      14          15      16      17      18      19      20   
Minimum  204404  204402      204398  204396  204394  204388  203986  203977   
Maximum  204404  204402      204400  204396  204394  204388  203986  203977   
Change   C -> A  K -> G  CAY -> GCT  A -> C  C -> A  C -> A  A -> T  C -> T   

sample                                                 
gene                                                   
number       21      22          23        24      25  
Minimum  203713  203705      203696    203679  203677  
Maximum  203713  203705      203698    203680  203677  
Change   A -> T  G -> A  ATC -> GTT  TG -> AA  C -> T

In [290]:
pd.merge(left=df, right=df2)

,Minimum,Maximum,Change
0,204538,204538,A -> G
1,204529,204529,G -> A
2,204459,204460,WA -> TT
3,204456,204456,A -> T
4,204453,204454,CG -> TT
5,204407,204408,CH -> AT
6,204404,204404,C -> A
7,204402,204402,K -> G
8,204398,204400,CAY -> GCT
9,204396,204396,A -> C


In [291]:
df_c = pd.concat([df, df2]) 

In [292]:
pd.concat([df_c.xs("C1"), df_c.xs("C2")], axis=1)

Minimum   Maximum            Change  Minimum  Maximum  \
gene     number                                                           
FLO1 CDS 0       204538.0  204538.0            A -> G   204538   204538   
         1       204529.0  204529.0            G -> A   204529   204529   
         2       204459.0  204460.0          WA -> TT   204459   204460   
         3       204456.0  204456.0            A -> T   204456   204456   
         4       204453.0  204454.0          CG -> TT   204453   204454   
         5       204449.0  204451.0        CAC -> GTT   204439   204441   
         6       204443.0  204448.0  TSSCCA -> GAAACC   204434   204436   
         7       204435.0  204435.0            G -> C   204431   204433   
         8       204431.0  204433.0        ACM -> CAA   204428   204430   
         9       204428.0  204430.0       BHB -> CGGT   204416   204418   
         10      204424.0  204426.0        RMS -> CCA   204413   204414   
         11      204419.0  204419.0            C -> A   204410   204412   
         12      204410.0  204411.0          CA -> AT   204407   204408   
         13      204407.0  204408.0          CH -> AT   204404   204404   
         14      204404.0  204404.0            C -> A   204402   204402   
         15      204402.0  204402.0            K -> G   204398   204400   
         16      204398.0  204400.0        CAY -> GCT   204396   204396   
         17      204396.0  204396.0            A -> C   204394   204394   
         18      204394.0  204394.0            C -> A   204388   204388   
         19      204388.0  204388.0            C -> A   203986   203986   
         20      203986.0  203986.0            A -> T   203977   203977   
         21      203977.0  203977.0            C -> T   203713   203713   
         22      203679.0  203680.0          TG -> AA   203705   203705   
         23      203677.0  203677.0            C -> T   203696   203698   
         24           NaN       NaN               NaN   203679   203680   
         25           NaN       NaN               NaN   203677   203677   

                     Change  
gene     number              
FLO1 CDS 0           A -> G  
         1           G -> A  
         2         WA -> TT  
         3           A -> T  
         4         CG -> TT  
         5       GWG -> ACT  
         6       CGA -> AAC  
         7       ACM -> GTC  
         8       BHB -> ACG  
         9       MMY -> TCA  
         10        AM -> CC  
         11      CAY -> ATA  
         12        CH -> AT  
         13          C -> A  
         14          K -> G  
         15      CAY -> GCT  
         16          A -> C  
         17          C -> A  
         18          C -> A  
         19          A -> T  
         20          C -> T  
         21          A -> T  
         22          G -> A  
         23      ATC -> GTT  
         24        TG -> AA  
         25          C -> T

In [302]:
pd.merge(left=df, right=df2, on=["Change", "Minimum", "Maximum"], right_index=True)

Minimum  Maximum      Change
sample gene     number                              
C1     FLO1 CDS 0        204538   204538      A -> G
                1        204529   204529      G -> A
                2        204459   204460    WA -> TT
                3        204456   204456      A -> T
                4        204453   204454    CG -> TT
                13       204407   204408    CH -> AT
                14       204404   204404      C -> A
                15       204402   204402      K -> G
                16       204398   204400  CAY -> GCT
                17       204396   204396      A -> C
                18       204394   204394      C -> A
                19       204388   204388      C -> A
                20       203986   203986      A -> T
                21       203977   203977      C -> T
                22       203679   203680    TG -> AA
                23       203677   203677      C -> T

In [309]:
df["Sample"] = pd.Series(len(df) * ["C1"])

In [317]:
df.drop("Sample", inplace=True, axis=1)

In [342]:
for i in df_c.index.levels:
    print list(i)

['C1', 'C2']
['FLO1 CDS']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]


In [344]:
df.xs("C1")

Minimum  Maximum            Change
gene     number                                    
FLO1 CDS 0        204538   204538            A -> G
         1        204529   204529            G -> A
         2        204459   204460          WA -> TT
         3        204456   204456            A -> T
         4        204453   204454          CG -> TT
         5        204449   204451        CAC -> GTT
         6        204443   204448  TSSCCA -> GAAACC
         7        204435   204435            G -> C
         8        204431   204433        ACM -> CAA
         9        204428   204430       BHB -> CGGT
         10       204424   204426        RMS -> CCA
         11       204419   204419            C -> A
         12       204410   204411          CA -> AT
         13       204407   204408          CH -> AT
         14       204404   204404            C -> A
         15       204402   204402            K -> G
         16       204398   204400        CAY -> GCT
         17       204396   204396            A -> C
         18       204394   204394            C -> A
         19       204388   204388            C -> A
         20       203986   203986            A -> T
         21       203977   203977            C -> T
         22       203679   203680          TG -> AA
         23       203677   203677            C -> T

In [268]:
df1 = W303_cog7_day42_C1_dfs["FLO1 CDS"]
df2 = W303_cog7_day42_C2_dfs["FLO1 CDS"]
df3 = W303_cog7_day42_C3_dfs["FLO1 CDS"]

In [269]:
for i in (df1, df2, df3):
    i.index = range(len(i))

In [271]:
df1.index, df2.index, df3.index

(Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
             17, 18, 19, 20, 21, 22, 23],
            dtype='int64'),
 Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
             17, 18, 19, 20, 21, 22, 23, 24, 25],
            dtype='int64'),
 Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
             17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
            dtype='int64'))

In [274]:
pd.concat([df1, pd.Series(["C1"] * len(df1))], axis=1)

,Minimum,Maximum,Change,0
0,204538,204538,A -> G,C1
1,204529,204529,G -> A,C1
2,204459,204460,WA -> TT,C1
3,204456,204456,A -> T,C1
4,204453,204454,CG -> TT,C1
5,204449,204451,CAC -> GTT,C1
6,204443,204448,TSSCCA -> GAAACC,C1
7,204435,204435,G -> C,C1
8,204431,204433,ACM -> CAA,C1
9,204428,204430,BHB -> CGGT,C1
